In [ ]:
#Install Dependencies
!pip install qiskit

In [ ]:
!pip install qiskit_machine_learning

In [ ]:
#Import Dependencies
import matplotlib.pyplot as plt
import numpy as np

from qiskit import Aer
from qiskit.circuit.library import ZZFeatureMap
from qiskit.utils import QuantumInstance, algorithm_globals
from qiskit_machine_learning.algorithms import QSVC
from qiskit_machine_learning.kernels import QuantumKernel
#import the noise packages
from qiskit.providers.aer.noise import NoiseModel
from qiskit.providers.aer.noise import QuantumError, ReadoutError
from qiskit.providers.aer.noise import pauli_error
from qiskit.providers.aer.noise import depolarizing_error,amplitude_damping_error,phase_damping_error,phase_amplitude_damping_error
from qiskit.providers.aer.noise import thermal_relaxation_error
seed = 12345
algorithm_globals.random_seed = seed

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA

In [ ]:
# Load the CSV file
df = pd.read_csv("/content/drive/MyDrive/image_data_final_quantum (2).csv")
d = df
X = d.drop(['class'], axis=1)

# Apply PCA to extract 10 features
pca = PCA(n_components=10)
X = pca.fit_transform(X)

# Target variable
y = d['class']

# Perform train-test split
train_features, test_features, train_labels, test_labels = train_test_split(X, y, test_size=0.25, random_state=42)


In [ ]:
#Quantum Instance Initialization and Feature Map Conversion
customdata_dimension = 10

feature_map = ZZFeatureMap(feature_dimension=customdata_dimension, reps=2, entanglement="linear")

backend = QuantumInstance(
    Aer.get_backend("aer_simulator"), shots=1024, seed_simulator=seed, seed_transpiler=seed)

kernel = QuantumKernel(feature_map=feature_map, quantum_instance=backend)

In [ ]:
#Initialize QSVC model
qsvc = QSVC(quantum_kernel=kernel)

In [ ]:
# Import time module
import time
 # record start time
start = time.time()

#Fit the model
qsvc.fit(train_features, train_labels)

end = time.time()

print("The time of execution of above program is :",
      (((end-start) * 10**3)/1000)/60, "mins")

The time of execution of above program is : 95.75489753087362 mins


In [ ]:
#Model prediction
predicted_labels_test = qsvc.predict(test_features)
predicted_labels_train = qsvc.predict(train_features)

In [ ]:
#Calculate accuracy
from sklearn.metrics import accuracy_score
print(f'Quantum QSVC (sklearn) test accuracy {accuracy_score(test_labels,predicted_labels_test):}')
print(f'Quantum QSVC (sklearn) train accuracy {accuracy_score(train_labels,predicted_labels_train):}')

In [ ]:
#Save the model
qsvc.save("/content/drive/MyDrive/QSVC_MODEL_SAVE")